We show how to compute the symmetrized product of two symmetric matrices $A$, $B$, defined by $C=AB+BA$ using ${n+2 \choose 3}$ multiplications rather than $n^3$. Some third order intermediate tensors are computed using the Cyclops Tensor Framework (CTF) library

In [1]:
import numpy as np
import ctf

Generate random symmetric matrices and compute a reference answer using `numpy`.

In [2]:
n=4
def get_rand_sym_mat(n):
    A = np.random.random((n,n))-.5
    A = np.triu(A)
    A = A + A.T
    return A
A = get_rand_sym_mat(n)
B = get_rand_sym_mat(n)

C_ref = A @ B + B @ A

Compute intermediate tensor $$\hat{Z}_{ijk} = (A_{ij}+A_{ik}+A_{jk})\cdot(B_{ij}+B_{ik}+B_{jk})$$

In [3]:
A = ctf.astensor(A)
B = ctf.astensor(B)
C = ctf.tsr([n,n])

Zhat = ctf.tsr([n,n,n])
Zhat.i("ijk")<<(A.i("ij")+A.i("ik")+A.i("jk"))*(B.i("ij")+B.i("ik")+B.i("jk"))

Write a function to verify symmetry of tensors and apply it to $\hat{Z}$. We verify symmetry by checking that any transposition of modes leads to the same tensor. For a third order tensor, this implies that it has ${n+2\choose 3}$ unique elements. We will avoid accessing the diagonals of $\hat{Z}$, meaning they do not have to be computed, and only ${n\choose 3}$ elements of $\hat{Z}$ will be computed.

In [4]:
def verify_sym(A):
    for i in range(A.ndim):
        for j in range(i):
            istr1 = np.asarray(range(A.ndim))
            istr2 = istr1.copy()
            istr2[i] = istr1[j]
            istr2[j] = istr1[i]
            str1 = ''.join(str(s) for s in istr1)
            str2 = ''.join(str(s) for s in istr2)
            B = ctf.tsr(A.shape)
            B.i(str1) << A.i(str1)
            B.i(str2) << -1.0*B.i(str1)
            assert(B.norm2()/B.size < .000001)
#verify symmetry
verify_sym(A)
verify_sym(Zhat)
#zero out diagonal elements we will not use
Zhat.i("jii").scl(0.0) 
Zhat.i("iji").scl(0.0) 
Zhat.i("iij").scl(0.0)
#print(Zhat)
print("Number of unique nonzeros in Zhat is now", np.count_nonzero(Zhat.to_nparray())/6)
print("This is also the number of multiplications we need to compute Zhat")

Number of unique nonzeros in Zhat is now 4.0
This is also the number of multiplications we need to compute Zhat


Now compute a partial sum of $\hat{Z}$ into $Z$.

In [5]:
Z = ctf.tsr(A.shape)
Z.i("ij") << Zhat.i("ijk")
#Z

We define a function `a = j(A)` which computes
$$a_i = A_{ii} - \sum_{j\neq i} A_{ij}$$

In [6]:
def j(A):
    B = ctf.tsr([n])
    B.i("i") << -1.0*A.i("ij") + 2.0*A.i("ii")
    return B
a = j(A)
b = j(B)
def npj(A):
    B = np.zeros(n);
    B[:] -= np.sum(A[:,i] for i in range(n))
    B += 2.*np.diagonal(A)
    return B
print(j(A))
print(npj(A.to_nparray()))
print(npj(A.to_nparray()) @ npj(B.to_nparray())-np.sum(npj(A.to_nparray()*npj(B.to_nparray()))))
print(npj(A.to_nparray()) @ npj(B.to_nparray())-np.sum(npj(A.to_nparray()*B.to_nparray())))
#print(A)
#a

array([-1.352427  , -1.85080181, -0.35072607,  0.31126246])
[-1.352427   -1.85080181 -0.35072607  0.31126246]
-3.05311331772e-16
0.566246881446


Compute matrix
$$U_{ij} = (A_{ij} + a_i + a_j)\cdot (B_{ij} + b_i + b_j)$$
where $a$ and $b$ are jacobinated $A$ and $B$. The elements of $U$ are meant to cancel some unwated terms in corresponding element of $Z$, but also introduce some new unwanted terms. Only $n \choose 2$ multiplications are needed to compute $U$, since it is symmetric and we will not use its diagonal.

In [7]:
U = ctf.tsr(A.shape)
U.i("ij") << (A.i("ij") + a.i("i") + a.i("j"))*(B.i("ij") + b.i("i") + b.i("j"))
U.i("ii").scl(0.0)
#U

Compute matrix
$$V_{ij} = (a_i + a_j)\cdot (b_i + b_j)$$
where $a$ and $b$ are jacobinated $A$ and $B$. The elements of $V$ are meant to cancel the unwated terms in corresponding element of $U$. Only $n \choose 2$ multiplications are needed to compute $V$, since it is symmetric and we will not use its diagonal.

In [11]:
V = ctf.tsr(A.shape)
V.i("ij") << (a.i("i") + a.i("j"))*(b.i("i") + b.i("j"))
V.i("ii").scl(0.0)
V

array([[ 0.        ,  1.32653264,  1.06539428,  0.41765836],
       [ 1.32653264, -0.        , -3.44133795, -2.75201252],
       [ 1.06539428, -3.44133795, -0.        , -0.06220005],
       [ 0.41765836, -2.75201252, -0.06220005,  0.        ]])

Now, lastly we need a vector of the form $$w_{i}=\sum_{j\neq i} A_{ij}\cdot B_{ij}$$ to cancel the remaining terms appearing in $Z_{ij}-U_{ij}+V_{ij}$ that we do not want, i.e. we want $w_{i} + w_{j} = Z_{ij} - U_{ij} + V_{ij}$. We could compute $w$ directly via $n \choose 2$ multiplications. Instead, we get it from $U$ and $V$ via only $n$ multiplications, as we can show that 
$$\sum_{j\neq i} U_{ij}-V_{ij} = w_i + \sum_{j\neq i} \Big[A_{ij}(b_i + b_j) + (a_i+a_j)B_{ij}\Big].$$
We can simplify the latter unwanted term as
$$\sum_{j\neq i} (A_{ii}-a_{i})b_i + A_{ij}b_j + a_i(B_{ii}-b_i)+ a_jB_{ij}$$
Applying the identity $<j(A)j(B)> = \sum j(A\odot j(B))$, we get (FIXME: things don't quite cancel, adjust scaling in j(A))
$$w_i - \sum_{j\neq i} U_{ij}-V_{ij} = (n-1)(A_{ii}b_i + a_iB_{ii})= (n-1)(A_{ii}+a_i)(b_i+B_{ii}) - (n-1)A_{ii}B_{ii}.$$
Therefore we can compute
$$w_i + frac{n-1}2(U_{ii}-V_{ii})-\sum_{j\neq i} (U_{ij}-V_{ij})=\frac{n-1}2(2A_{ii}b_i+2B_{ii}a_i) - ((n-1)(A_{ii}+a_i)(b_i+B_{ii})-  (n-1)A_{ii}B_{ii}) =  (n-1)(A_{ii}+a_i)(b_i+B_{ii}).$$
We then need only to compute $n$ multiplications $(A_{ii}+a_i)(b_i+B_{ii})$.

In [10]:
Vref = ctf.tsr([n])
V = ctf.tsr([n])
Vref.i("i") << A.i("ij")*B.i("ij")
V.i("i") << U.i("ij")+-1.0*V.i("ij")
V.i("i") << -(n-1.)*(A.i("ii")+a.i("i"))*(B.i("ii")+b.i("i"))
print(V)
print(Vref)

array([ -3.7748156 ,  18.64790328,   7.42868586,   2.94732615])
array([ 0.65984785, -0.88720597, -0.00808042,  0.23309865])
